Plot the surface temperature field from the IBI model (CMEMS)

In [17]:
import sys
sys.path.insert(0, '..')
import os
import glob
import netCDF4
from filament import filament
import datetime
import cmocean
import logging
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cartopyticker
myproj = ccrs.PlateCarree()
coast = cfeature.GSHHSFeature(scale="f")
plt.style.use("./filament.mplstyle")

In [18]:
logger = logging.getLogger("filamentIBIPlot")
logger.setLevel(logging.INFO)
logging.info("Starting")
plotvis = 0

In [19]:
datadir = "/home/ctroupin/data/CMEMS/IBI_ANALYSISFORECAST_PHY_005_001/"
figdir = "../figures/currents"
if not(os.path.isdir(figdir)):
    os.mkdir(figdir)

datafilelist = sorted(glob.glob(os.path.join(datadir, "*.nc")))
nfiles = len(datafilelist)
logger.info("Processing {} files".format(nfiles))
canarydomain = (-18.5, -13., 26., 29.5)

INFO:filamentIBIPlot:Processing 5 files


## Load data

In [ ]:
sst = filament.SST()
sst.read_from_cmems(datafilelist[0], 0)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = plt.subplot(111, projection=myproj)
ax.pcolor(sst.lon, sst.lat, sst.field)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = plt.subplot(111, projection=myproj)
#plt.title("CMEMS dataset-ibi-analysis-forecast-phys-005-001-hourly")

pcm, cb = sst.add_to_plot(fig, ax, canarydomain, cmap=plt.cm.RdYlBu_r,
                    clim=[21., 24.], vis=False,
                    cbarloc=[0.575, 0.275, 0.3, 0.02], alpha=1)
sst.add_date(ax)
cb.set_ticks(np.arange(21., 24.01, 0.5))
ax.add_feature(coast, linewidth=.2, color=".5")
filament.decorate_map(ax, canarydomain, np.arange(-20., -10., 2.), 
                      np.arange(25., 32., 1.))
#plt.savefig(os.path.join(figdir, "SST_CMEMS_20200608"))
plt.show()

## Compare with SST from satellite

In [ ]:
sstfile = "/data/OceanColor/SNPP_VIIRS.20200608T140000.L2.SST.NRT.nc"
sstsat = filament.SST()
sstsat.read_from_oceancolorL2(sstfile)
sstsat.field = np.ma.masked_where(sstsat.qflag > 2, sstsat.field)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = plt.subplot(111, projection=myproj)
plt.title("SNPP VIIRS")

pcm, cb = sstsat.add_to_plot(fig, ax, canarydomain, cmap=plt.cm.RdYlBu_r,
                    clim=[21., 24.], vis=False,
                    cbarloc=[0.575, 0.275, 0.3, 0.02], alpha=1)
sstsat.add_date(ax)
cb.set_ticks(np.arange(21., 24.01, 0.5))
ax.add_feature(coast, linewidth=.2, color=".5")
filament.decorate_map(ax, canarydomain, np.arange(-20., -10., 2.), 
                      np.arange(25., 32., 1.))
plt.savefig(os.path.join(figdir, "SST_VIIRS_20200608"))
plt.show()
plt.close()

## Velocity field

In [37]:
def read_velocity(datafile):
    with netCDF4.Dataset(datafile) as nc:

        lat = nc.variables["latitude"][:]
        lon = nc.variables["longitude"][:]
        time = nc.variables["time"]
        timeunits = time.units
        dates = netCDF4.num2date(time[:], timeunits)
        # depth = nc.variables["depth"][:]
        goodlon = np.where( (lon >= canarydomain[0]) & (lon <= canarydomain[1]) )[0]
        goodlat = np.where( (lat >= canarydomain[2]) & (lat <= canarydomain[3]) )[0]
        uvel = nc.variables["uo"][:,0,goodlat,goodlon]
        vvel = nc.variables["vo"][:,0,goodlat,goodlon]
        lon = lon[goodlon]
        lat = lat[goodlat]

    llon, llat = np.meshgrid(lon, lat)
    return llon, llat, uvel, vvel, dates

In [38]:
def plot_speed(lon, lat, u, v, date=None, NN=2, figname=None):
    
    datestring = date.strftime()
    speed = np.sqrt(u * u + v * v)
    
    fig = plt.figure(figsize=(12, 10))
    ax = plt.subplot(111, projection=myproj)
    
    if date is not None:
        plt.title(f"Surface currents on {datestring}\n(CMEMS IBI model)", fontsize=24)
        
    q = plt.quiver(lon[::NN,::NN], lat[::NN,::NN], u[::NN,::NN]/ speed[::NN,::NN], 
                   v[::NN,::NN]/ speed[::NN,::NN], 
               speed[::NN,::NN], cmap=cmocean.cm.speed, scale=100, width=.001)
    ax.add_feature(coast, linewidth=.2, color=".5")
    filament.decorate_map(ax, canarydomain, np.arange(-20., -10., 2.), 
                          np.arange(25., 32., 1.))
    cb = plt.colorbar(extend="max", shrink=.5)
    cb.set_label("v (m/s)", rotation=0, ha='left')
    cb.set_ticks(np.arange(0., 0.501, 0.1))
    plt.clim(0., 0.5)
    if figname is not None:
        plt.savefig(figname)
    plt.close(fig)

In [39]:
llon, llat, uvel, vvel, dates = read_velocity(datafilelist[0])

In [40]:
fname1 = os.path.basename(datafilelist[0]).split(".")[0]
for iid, ddd in enumerate(dates):
    print(iid)
    fname2 = ddd.strftime("%Y%m%d_%H%M%S")
    figname = os.path.join(figdir, "_".join((fname1, fname2)))
    plot_speed(llon, llat, uvel[iid,:,:], vvel[iid,:,:], ddd, NN=1, figname=figname)

0


/tmp/ipykernel_231813/1440231027.py:4: RuntimeWarning: invalid value encountered in sqrt
  speed = np.sqrt(u * u + v * v)


In [41]:
ddd

cftime.DatetimeGregorian(2023, 3, 21, 12, 0, 0, 0, has_year_zero=False)